In [15]:
import pandas as pd
from thefuzz import fuzz

# Default columns for the municipality dataframe
default_columns_by_municipality = ["Gemeinde", "Kanton", "Bezirk"]

# Metadata columns of interest
columns_metadata = ["Title (de)", "Date", "Institution", "Theme 1", "Theme 2", 
                    "Theme 3", "Vote trigger", "Vote Result status", 
                    "Official status", "Legal act type", "Vote trigger actor"]

# Load data
df_metadata = pd.read_csv("cleaned_metadata_per_votum.csv")
df_municipality = pd.read_csv("cleaned_data_by_municipality.csv")

# Metadata dictionary for storing extracted metadata
d = {}

for column in df_municipality.columns:
    if column not in default_columns_by_municipality:
        abstimmung_datum = column[:10]
        abstimmung_name = column[11:]
        metadata_by_date = df_metadata[df_metadata["Date"] == abstimmung_datum]
        
        # Calculate similarity score
        df_metadata['similarity_score'] = df_metadata["Title (de)"].apply(
            lambda title: fuzz.partial_ratio(title, abstimmung_name)
        )
        metadata_by_name = metadata_by_date[df_metadata['similarity_score'] > 85]
        
        # If exactly one match, store metadata
        if metadata_by_name.shape[0] == 1:
            d[column] = metadata_by_name.iloc[0]

# Create metadata rows aligned to df_municipality's columns
metadata_rows = {
    "Institution": [],
    "Theme 1": [],
    "Legal act type": [],
    "Vote trigger actor": [],
    "Vote Result status": []
}

# Populate metadata values
for column in df_municipality.columns:
    if column in default_columns_by_municipality:
        # Non-voting columns
        for key in metadata_rows.keys():
            metadata_rows[key].append(None)
    else:
        # Voting columns
        if column in d:
            metadata_rows["Institution"].append(d[column].get("Institution", None))
            metadata_rows["Theme 1"].append(d[column].get("Theme 1", None))
            metadata_rows["Legal act type"].append(d[column].get("Legal act type", None))
            metadata_rows["Vote trigger actor"].append(d[column].get("Vote trigger actor", None))
            metadata_rows["Vote Result status"].append(d[column].get("Vote Result status", None))
        else:
            # No metadata found
            for key in metadata_rows.keys():
                metadata_rows[key].append(None)

# Create DataFrame for metadata rows
metadata_df = pd.DataFrame(metadata_rows)
metadata_df.index = ["Institution", "Theme 1", "Legal act type", 
                     "Vote trigger actor", "Vote Result status"]

# Transpose metadata to match df_municipality columns
metadata_df = metadata_df.T
metadata_df.columns = df_municipality.columns  # Align columns

# Combine metadata rows and original data
df_municipality_with_metadata = pd.concat([metadata_df, df_municipality], ignore_index=True)

# Save or display the resulting DataFrame
df_municipality_with_metadata.to_csv("df_municipality_with_metadata.csv", index=False)


The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_16867/3850145426.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_by_name = metadata_by_date[df_metadata['similarity_score'] > 85]
/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_16867/3850145426.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_by_name = metadata_by_date[df_metadata['similarity_score'] > 85]
/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_16867/3850145426.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_by_name = metadata_by_date[df_metadata['similarity_score'] > 85]
/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_16867/3850145426.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_by_name = metadata_by_date[df_metadata['similarity_score'] > 85]
/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_1

ValueError: Length mismatch: Expected axis has 495 elements, new values have 5 elements